In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
%matplotlib inline

In [70]:
raw_training_data = pd.read_csv('./data/results_training_v3.csv')
raw_labels = pd.read_csv('./data/training_fluid_intelligenceV1.csv')
raw_description_data = pd.read_csv('./data/btsv01.txt', sep='\t')
merge_with_description = True
#now we merge it 

if merge_with_description: 
    raw_training_data = pd.merge(raw_training_data, raw_description_data, left_on='subject', right_on='subjectkey')
    raw_training_data = raw_training_data.drop([0], axis=0)
    raw_training_data = raw_training_data.drop(['btsv01_id', 'interview_date', 'collection_id', \
                              'dataset_id', 'collection_title', 'src_subject_id', 'gender', 'subjectkey'], axis=1)

train_data_with_labels = pd.merge(raw_training_data, raw_labels, on='subject')
labels = train_data_with_labels['residual_fluid_intelligence_score']


train_data = train_data_with_labels.drop(['subject', 'residual_fluid_intelligence_score'], axis=1)

/Users/admin/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (0,1,2,6,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['sri24cblmhemiwhtrwm', 'sri24ponswm', 'sri24corpuscallosumwm',
       'sri24wm400wm400lwm', 'sri24wm400wm400rwm', 'sri24vtlslateralvtllcsf',
       'sri24vtlslateralvtlrcsf', 'sri24vtlsthirdvtllcsf',
       'sri24vtlsthirdvtlrcsf', 'sri24suptentsupratentoriumv'],
      dtype='object')


In [71]:
#first convert to matrix and normalize data
def normalize_data(data: np.array, standardize=True):
    means = np.mean(data, axis=0)
    stds = np.std(data, axis=0)
    if not standardize:
        return data-means
    return (data-means)/stds

train_data = normalize_data(train_data.values.astype(float))

In [80]:
#now explore the models and cross-val 
from sklearn.model_selection import cross_validate
def mse(real, pred):
    print(real.shape, pred.shape)
    return np.mean(np.square(real-pred))

def run_crossval(clf, X, y,n_cv=3):
    cv_scores = cross_validate(clf, X,y, cv=n_cv, scoring='neg_mean_squared_error')
    return cv_scores

In [ ]:
from sklearn.svm import SVR 

svr = SVR()

np.mean(run_crossval(svr, train_data, labels, 10)['test_score'])

In [79]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_data, labels, test_size=0.1)
svr.fit(X_train, y_train)
xxx= svr.predict(X_test)
mse(y_test, xxx)

(187,) (187,)


86.77449157539301